In [95]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
#from ydata_profiling import ProfileReport
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import xgboost as xgb
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.metrics import roc_curve, auc
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.metrics import log_loss, brier_score_loss
from sklearn.calibration import calibration_curve
from scipy.special import expit  # Sigmoid function
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
from sklearn.metrics import (
    confusion_matrix, 
    classification_report, 
    roc_auc_score, 
    precision_recall_curve, 
    average_precision_score,
    f1_score,
    log_loss
)
from joblib import Parallel, delayed
import joblib

import plotly.express as px
import os
from PyPDF2 import PdfMerger


In [96]:
df = pd.read_csv(r'/Users/selinsevim/Desktop/breast_cancer_prediction_ai_model/data/raw/data.csv')

In [97]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [98]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [99]:
df.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,0.0
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946,NaN
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061,NaN
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,NaN
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460,NaN
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040,NaN
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080,NaN
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,NaN


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [101]:
#report = ProfileReport(df)

In [102]:
#report

In [103]:
# Check if there are Null values in the dataframe
df.isna().sum()

id                           0
diagnosis                    0
radius_mean                  0
texture_mean                 0
perimeter_mean               0
area_mean                    0
smoothness_mean              0
compactness_mean             0
concavity_mean               0
concave points_mean          0
symmetry_mean                0
fractal_dimension_mean       0
radius_se                    0
texture_se                   0
perimeter_se                 0
area_se                      0
smoothness_se                0
compactness_se               0
concavity_se                 0
concave points_se            0
symmetry_se                  0
fractal_dimension_se         0
radius_worst                 0
texture_worst                0
perimeter_worst              0
area_worst                   0
smoothness_worst             0
compactness_worst            0
concavity_worst              0
concave points_worst         0
symmetry_worst               0
fractal_dimension_worst      0
Unnamed:

In [104]:
df = df.drop(columns = "Unnamed: 32", axis = 1) 

In [105]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [106]:
df.isnull().any()

id                         False
diagnosis                  False
radius_mean                False
texture_mean               False
perimeter_mean             False
area_mean                  False
smoothness_mean            False
compactness_mean           False
concavity_mean             False
concave points_mean        False
symmetry_mean              False
fractal_dimension_mean     False
radius_se                  False
texture_se                 False
perimeter_se               False
area_se                    False
smoothness_se              False
compactness_se             False
concavity_se               False
concave points_se          False
symmetry_se                False
fractal_dimension_se       False
radius_worst               False
texture_worst              False
perimeter_worst            False
area_worst                 False
smoothness_worst           False
compactness_worst          False
concavity_worst            False
concave points_worst       False
symmetry_w

In [107]:
df["diagnosis"] = df["diagnosis"].map({'M':1, 'B':0})

In [108]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [109]:
# Map numeric diagnosis back to string labels for plotting
diagnosis_labels = df['diagnosis'].map({0: 'Benign', 1: 'Malignant'})

fig1 = px.histogram(x=diagnosis_labels, 
                   color=diagnosis_labels,
                   color_discrete_map={
        'Benign': 'rgb(50, 80, 168)',     
        'Malignant': '#ee8ef5'     
    })
fig1.update_xaxes(categoryorder="array", categoryarray=['Benign', 'Malignant'])
fig1.update_layout(
    title="Distribution of Diagnosis",
    xaxis_title="Diagnosis",
    yaxis_title="Count of Diagnosis Distribution"
)
fig1.update_layout(
    width=600,
    height=400,
    font=dict(
        family="Arial, Courier New, monospace",  # font family
        size=12,                                 # font size (pixels)
        color="darkblue"                         # font color
    )
)

fig1.update_layout(
    plot_bgcolor='white',
    xaxis=dict(
        showline=True,            # show axis line
        linecolor='darkgray',        # axis line color
        showgrid=True,            # show grid lines
        gridcolor='lightgray',    # grid line color
        gridwidth=1
    ),
    yaxis=dict(
        showline=True,
        linecolor='darkgray',
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=1
    )
)

fig1.show()

# filename = f"fig_1.pdf"
# fig1.write_image(filename)

In [110]:
df.corr()['diagnosis'] > 0

id                          True
diagnosis                   True
radius_mean                 True
texture_mean                True
perimeter_mean              True
area_mean                   True
smoothness_mean             True
compactness_mean            True
concavity_mean              True
concave points_mean         True
symmetry_mean               True
fractal_dimension_mean     False
radius_se                   True
texture_se                 False
perimeter_se                True
area_se                     True
smoothness_se              False
compactness_se              True
concavity_se                True
concave points_se           True
symmetry_se                False
fractal_dimension_se        True
radius_worst                True
texture_worst               True
perimeter_worst             True
area_worst                  True
smoothness_worst            True
compactness_worst           True
concavity_worst             True
concave points_worst        True
symmetry_w

In [111]:
# Map numeric diagnosis back to string labels for plotting
diagnosis_labels = df['diagnosis'].map({0: 'Benign', 1: 'Malignant'})

In [112]:
fig2 = px.scatter_matrix(df,
    dimensions=['radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
    color=diagnosis_labels, 
    color_discrete_map={
        'Benign': 'rgb(50, 80, 168)',     
        'Malignant': '#ee8ef5'     
    })

fig2.update_layout(
    title=dict(text='Breast Cancer Diagnosis'),
    width=1500,
    height=1500,
    font=dict(
        family="Arial, Courier New, monospace",  # font family
        size=12,                                 # font size (pixels)
        color="darkblue"                         # font color
    ),
    plot_bgcolor='white'
)

fig2.show()

# filename = f"fig_2.pdf"
# fig2.write_image(filename)

In [113]:
# Benign cases: Peak at lower values (left side)
# Malignant cases: Peak at higher values (right side)
# Minimal overlap in the middle

In [114]:
# Both groups: Similar peaks at similar locations
# Heavy overlap throughout the range
# Hard to distinguish the two colors

Clear Separation = Easy Classification:

When histograms show distinct peaks with minimal overlap
A simple threshold could classify most cases correctly
High accuracy expected from machine learning models

Heavy Overlap = Difficult Classification:

When both diagnosis groups have similar distributions
High error rates expected
Models will struggle to distinguish between classes

Threshold-Based Classification:
If you see clear separation (like area_worst), you could literally draw a vertical line:
If area_worst > 800: Predict Malignant
If area_worst ≤ 800: Predict Benign
This simple rule might achieve 85-90% accuracy!

Features with good separation → Keep these
good_features = ["area_worst", "radius_worst", "concavity_worst"]

Features with poor separation → Consider removing
weak_features = ["symmetry_mean", "fractal_dimension_mean"]

In [115]:
columns_to_plot = ['radius_mean', 'texture_mean', 'perimeter_mean']

for column in columns_to_plot:
    fig = px.histogram(df, x=df[column], color="diagnosis", marginal="box",
                       color_discrete_map={
                                0: 'rgb(50, 80, 168)',     
                                1: '#ee8ef5'})
    fig.update_layout(
    title=dict(text=f"Box plot of {column} by Diagnosis"),
    width=800,
    height=400
    )
    fig.update_layout(
    width=600,
    height=400,
    font=dict(
        family="Arial, Courier New, monospace",  # font family
        size=12,                                 # font size (pixels)
        color="darkblue"                         # font color
    )
    )

    fig.update_layout(
        plot_bgcolor='white',
        xaxis=dict(
            showline=True,            # show axis line
            linecolor='darkgray',        # axis line color
            showgrid=True,            # show grid lines
            gridcolor='lightgray',    # grid line color
            gridwidth=1
        ),
        yaxis=dict(
            showline=True,
            linecolor='darkgray',
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=1
        )
    )
    fig.show()
    
    # filename = f"fig_{column}.pdf"
    # fig.write_image(filename)

In [116]:
columns_to_plot = ['radius_mean', 'texture_mean', 'perimeter_mean']

for column in columns_to_plot:
    fig = px.box(df, x="diagnosis", 
                 y=column, 
                 color="diagnosis", 
                 color_discrete_map={
                                0: 'rgb(50, 80, 168)',     
                                1: '#ee8ef5'}
    )
    fig.update_layout(
        title=dict(text=f"Box plot of {column} by Diagnosis"),
        width=800,
        height=400
    )
    fig.update_layout(
    width=600,
    height=400,
    font=dict(
        family="Arial, Courier New, monospace",  # font family
        size=12,                                 # font size (pixels)
        color="darkblue"                         # font color
    )
    )

    fig.update_layout(
        plot_bgcolor='white',
        xaxis=dict(
            showline=True,            # show axis line
            linecolor='darkgray',        # axis line color
            showgrid=True,            # show grid lines
            gridcolor='lightgray',    # grid line color
            gridwidth=1
        ),
        yaxis=dict(
            showline=True,
            linecolor='darkgray',
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=1
        )
    )
    fig.show()
    # filename = f"temp_fig_{column}.pdf"
    # fig.write_image(filename)

In [117]:
corr_matrix = df.corr(numeric_only=True)
fig3 = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='Purp',  
    text=corr_matrix.values,  # Add the correlation values as text
    texttemplate="%{z:.2f}",  # Format the text (2 decimal places)
    showscale=True
))
fig3.update_layout(title='Correlation Heatmap')
fig3.update_layout(
    width=1000,
    height=800,
    font=dict(
        family="Arial, Courier New, monospace",  # font family
        size=12,                                 # font size (pixels)
        color="darkblue"                         # font color
    )
)
fig3.show()

# filename = f"fig_3.pdf"
# fig3.write_image(filename)

In [118]:
df_numeric = df.copy()
target_corr = df_numeric.corr()['diagnosis'].drop('diagnosis').sort_values(ascending=False)

print("Features most correlated with diagnosis:")
print(target_corr.head(10))

print("Features negatively correlated with diagnosis:")
print(target_corr.tail(10))

Features most correlated with diagnosis:
concave points_worst    0.793566
perimeter_worst         0.782914
concave points_mean     0.776614
radius_worst            0.776454
perimeter_mean          0.742636
area_worst              0.733825
radius_mean             0.730029
area_mean               0.708984
concavity_mean          0.696360
concavity_worst         0.659610
Name: diagnosis, dtype: float64
Features negatively correlated with diagnosis:
symmetry_mean              0.330499
fractal_dimension_worst    0.323872
compactness_se             0.292999
concavity_se               0.253730
fractal_dimension_se       0.077972
id                         0.039769
symmetry_se               -0.006522
texture_se                -0.008303
fractal_dimension_mean    -0.012838
smoothness_se             -0.067016
Name: diagnosis, dtype: float64


In [119]:
# Find features with correlation > 0.9 (or <-0.9)
threshold = 0.9
corr_pairs = corr_matrix.abs().unstack().sort_values(ascending=False)

# Filter out self-correlations
high_corr_pairs = [(a, b, corr_matrix.loc[a, b]) for a, b in corr_pairs.index if a != b and corr_matrix.loc[a, b] > threshold]

# Show results
for a, b, corr_val in high_corr_pairs:
    print(f"{a} & {b} => correlation: {corr_val:.3f}")

perimeter_mean & radius_mean => correlation: 0.998
radius_mean & perimeter_mean => correlation: 0.998
perimeter_worst & radius_worst => correlation: 0.994
radius_worst & perimeter_worst => correlation: 0.994
area_mean & radius_mean => correlation: 0.987
radius_mean & area_mean => correlation: 0.987
perimeter_mean & area_mean => correlation: 0.987
area_mean & perimeter_mean => correlation: 0.987
area_worst & radius_worst => correlation: 0.984
radius_worst & area_worst => correlation: 0.984
perimeter_worst & area_worst => correlation: 0.978
area_worst & perimeter_worst => correlation: 0.978
radius_se & perimeter_se => correlation: 0.973
perimeter_se & radius_se => correlation: 0.973
perimeter_worst & perimeter_mean => correlation: 0.970
perimeter_mean & perimeter_worst => correlation: 0.970
radius_worst & radius_mean => correlation: 0.970
radius_mean & radius_worst => correlation: 0.970
perimeter_mean & radius_worst => correlation: 0.969
radius_worst & perimeter_mean => correlation: 0.96

<strong>Geometric Relationships (0.98-0.99 correlations):</strong> <br>
Radius ↔ Perimeter: Mathematically linked (perimeter ≈ 2πr for circles)<br>
Radius ↔ Area: Geometrically linked (area ≈ πr² for circles) <br>
Perimeter ↔ Area: Both measure size, just different dimensions<br>

<strong>Measurement Consistency (0.91-0.97 correlations):</strong><br>
mean ↔ worst: Same feature, different summary statistics<br>
texture_mean ↔ texture_worst: Measuring same cell property at different extremes<br>
If a tumor has rough texture on average, it likely has very rough texture at worst points<br>

<strong>Feature	        Correlation	Interpretation</strong> <br>
concave points_worst	0.79	    High number of concave contour points in the worst case is a very strong indicator of malignancy. <br>
perimeter_worst	        0.78	    Larger perimeter suggests larger or more irregular cell shapes.<br>
concave points_mean	    0.77	    Even the average number of concave points is strongly predictive.<br>
radius_worst	        0.77	    Larger radius in worst case linked with malignant growth.<br>
perimeter_mean	        0.74	    Similar story — average perimeter is important.<br>
area_worst	            0.73	    Malignant tumors are often larger.<br>
radius_mean	            0.73	    Again, size matters here.<br>
area_mean	            0.71	    Strongly predictive.<br>
concavity_mean	        0.70	    Irregular shape is a key feature of malignancy.<br>
concavity_worst	        0.66	    Reinforces how irregular boundaries matter.<br>

In [120]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [121]:
df = df.drop(columns=['id'])

In [122]:
df.columns

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [123]:
df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

Splitting the dataset

In [124]:
X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

In [125]:
X.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [126]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: diagnosis, dtype: int64

In [127]:
from sklearn.model_selection import StratifiedKFold

In [128]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [129]:
fold_metrics = []

In [130]:
for fold, (train_idx, test_idx) in enumerate(cv.split(X, y)):
    print(f"Fold {fold + 1}")
    
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = xgb.XGBClassifier(
        objective = 'binary:logistic',
        eval_metric = 'logloss',
        random_state = 42,
        scale_pos_weight= 2
    )
    
    model.fit(X_train_scaled, y_train)
    
    # # Create a CalibratedClassifierCV
    # calibrated_model = CalibratedClassifierCV(model, method='sigmoid')
    # calibrated_model.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)], verbose=False)  # Fit the calibration

    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # Print the classification report and confusion matrix
    print(classification_report(y_test, y_pred))
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    
    cm = confusion_matrix(y_test, y_pred)
    cm_norm = cm / cm.sum(axis=1)[:, np.newaxis]
    
    fig_confusion_matrix = px.imshow(cm_norm, labels=dict(x="Predicted", y="True"), 
                                     x=['Benign', 'Malignant'], y=['Benign', 'Malignant'], text_auto=True, color_continuous_scale=[[0.0, '#ee8ef5'], [1.0, 'rgb(50, 80, 168)']])
    fig_confusion_matrix.update_layout(title="Confusion Matrix", width=500, height=500, font=dict(
        family="Arial, Courier New, monospace",  # font family
        size=12,                                 # font size (pixels)
        color="darkblue"                         # font color
    ))
    fig_confusion_matrix.show()
    
    # Compute fold metrics
    fold_accuracy = accuracy_score(y_test, y_pred)
    fold_precision = precision_score(y_test, y_pred)
    fold_recall = recall_score(y_test, y_pred)
    fold_f1 = f1_score(y_test, y_pred)
    fold_roc_auc = roc_auc_score(y_test, y_proba)
    fold_log_loss = log_loss(y_test, y_proba)
    fold_pr_auc = average_precision_score(y_test, y_proba)


    fold_metrics.append({'precision': fold_precision, 'recall': fold_recall, 'f1': fold_f1, 'pr_auc': fold_pr_auc, 'roc_auc': fold_roc_auc, 'accuracy': fold_accuracy, 'log_loss': fold_log_loss})

    print(f"Fold {fold + 1} metrics: Precision={fold_precision}, Recall={fold_recall}, F1={fold_f1}, PR AUC={fold_pr_auc}, ROC AUC = {fold_roc_auc}  Accuracy={fold_accuracy}, Log Loss={fold_log_loss}")

Fold 1
              precision    recall  f1-score   support

           0       0.99      0.97      0.98        71
           1       0.95      0.98      0.97        43

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114


Confusion Matrix:
[[69  2]
 [ 1 42]]


Fold 1 metrics: Precision=0.9545454545454546, Recall=0.9767441860465116, F1=0.9655172413793104, PR AUC=0.9945661787635697, ROC AUC = 0.9963969865705863  Accuracy=0.9736842105263158, Log Loss=0.07030360834550589
Fold 2
              precision    recall  f1-score   support

           0       0.93      0.97      0.95        71
           1       0.95      0.88      0.92        43

    accuracy                           0.94       114
   macro avg       0.94      0.93      0.93       114
weighted avg       0.94      0.94      0.94       114


Confusion Matrix:
[[69  2]
 [ 5 38]]


Fold 2 metrics: Precision=0.95, Recall=0.8837209302325582, F1=0.9156626506024096, PR AUC=0.9867601676194393, ROC AUC = 0.9911562397641664  Accuracy=0.9385964912280702, Log Loss=0.14266896641759186
Fold 3
              precision    recall  f1-score   support

           0       0.96      0.99      0.97        72
           1       0.97      0.93      0.95        42

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114


Confusion Matrix:
[[71  1]
 [ 3 39]]


Fold 3 metrics: Precision=0.975, Recall=0.9285714285714286, F1=0.9512195121951219, PR AUC=0.9873228492060029, ROC AUC = 0.9871031746031746  Accuracy=0.9649122807017544, Log Loss=0.10124771367856876
Fold 4
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        72
           1       0.91      1.00      0.95        42

    accuracy                           0.96       114
   macro avg       0.96      0.97      0.96       114
weighted avg       0.97      0.96      0.97       114


Confusion Matrix:
[[68  4]
 [ 0 42]]


Fold 4 metrics: Precision=0.9130434782608695, Recall=1.0, F1=0.9545454545454546, PR AUC=0.9978103292056781, ROC AUC = 0.9986772486772486  Accuracy=0.9649122807017544, Log Loss=0.07543131687550968
Fold 5
              precision    recall  f1-score   support

           0       0.97      0.99      0.98        71
           1       0.98      0.95      0.96        42

    accuracy                           0.97       113
   macro avg       0.97      0.97      0.97       113
weighted avg       0.97      0.97      0.97       113


Confusion Matrix:
[[70  1]
 [ 2 40]]


Fold 5 metrics: Precision=0.975609756097561, Recall=0.9523809523809523, F1=0.963855421686747, PR AUC=0.9869718954202267, ROC AUC = 0.9902749832327298  Accuracy=0.9734513274336283, Log Loss=0.1347677106523067


In [131]:
# Aggregate metrics
mean_precision = np.mean([m['precision'] for m in fold_metrics])
mean_recall = np.mean([m['recall'] for m in fold_metrics])
mean_f1 = np.mean([m['f1'] for m in fold_metrics])
mean_pr_auc = np.mean([m['pr_auc'] for m in fold_metrics])
mean_accuracy = np.mean([m['accuracy'] for m in fold_metrics])
mean_log_loss = np.mean([m['log_loss'] for m in fold_metrics])


print(f"\nCross-Validation Results:\nPrecision: {mean_precision:.4f}, Recall: {mean_recall:.4f},  F1 Score: {mean_f1:.4f}, PR AUC: {mean_pr_auc:.4f}, Accuracy: {mean_accuracy:.4f}, Log Loss: {mean_log_loss:.4f}")

# Create bar chart for accuracy, precision, recall, f1, and roc_auc
fig_metrics = px.bar(x=['Accuracy', 'Precision', 'Recall', 'F1', 'PR-AUC'], y=[mean_accuracy, mean_precision, mean_recall,mean_f1, mean_pr_auc], 
                labels={'x': 'Metric', 'y': 'Value'}, color=['Accuracy', 'Precision', 'Recall', 'F1', 'ROC-AUC'], 
                color_discrete_map={'Accuracy': '#ee8ef5', 'Precision': 'rgb(176, 255, 221)', 'Recall': 'rgb(50, 80, 168)', 'F1': 'rgb(254, 255, 186)', 'PR-AUC': 'rgb(163, 231, 240)'})

fig_metrics.update_layout(title="Model Metrics", width=1000, height=500)
fig_metrics.update_layout(font=dict(
        family="Arial, Courier New, monospace",  # font family
        size=15,                                 # font size (pixels)
        color="darkblue"                         # font color
    ))
fig_metrics.update_layout(plot_bgcolor='white')
fig_metrics.update_xaxes(
                mirror=True,
                ticks='outside',
                showline=True,
                linecolor='rgb(242,242,242)',
                gridcolor='rgb(242,242,242)'
            )
fig_metrics.update_yaxes(
                mirror=True,
                ticks='outside',
                showline=True,
                linecolor='rgb(242,242,242)',
                gridcolor='rgb(242,242,242)'
            )
fig_metrics.show()

# filename = f"fig_metrics.pdf"
# fig_metrics.write_image(filename)


Cross-Validation Results:
Precision: 0.9536, Recall: 0.9483,  F1 Score: 0.9502, PR AUC: 0.9907, Accuracy: 0.9631, Log Loss: 0.1049


In [132]:
features = X_train.columns.to_list()
print(f"Number of features: {len(features)}")
print(f"Number of feature importances: {len(model.feature_importances_)}")

# Ensure the lengths match
if len(features) == len(model.feature_importances_):
    feat_import = pd.DataFrame({'Feature': features, 'Feature importance': model.feature_importances_})
    feat_import = feat_import.sort_values(by='Feature importance', ascending=False)

    # Show with plotly

    fig4 = px.bar(feat_import, x='Feature', y='Feature importance', orientation='v', title='Feature Importance XGBoost', text_auto=True, color='Feature importance')
    fig4.update_layout(font=dict(
        family="Arial, Courier New, monospace",  # font family
        size=15,                                 # font size (pixels)
        color="darkblue"   ))
    fig4.update_layout(width=1000, height=700)
    fig4.update_layout(plot_bgcolor='white')
    fig4.update_xaxes(
                    mirror=True,
                    ticks='outside',
                    showline=True,
                    linecolor='rgb(242,242,242)',
                    gridcolor='rgb(242,242,242)'
                )
    fig4.update_yaxes(
                    mirror=True,
                    ticks='outside',
                    showline=True,
                    linecolor='rgb(242,242,242)',
                    gridcolor='rgb(242,242,242)')
    fig4.show()

else:
    print("The number of features and feature importances do not match.")

Number of features: 30
Number of feature importances: 30


In [133]:
step_factor = 0.05
threshold_value = 0.2
recall = 0

pred_proba = model.predict_proba(X_test_scaled)
while threshold_value <= 1.0:
    temp_threshold = threshold_value
    predicted = (pred_proba[:,1] >= threshold_value).astype('int')
    print('Threshold:', temp_threshold, '--- Recall Score:', recall_score(y_test, predicted))
    if recall<recall_score(y_test, predicted):
        recall = recall_score(y_test, predicted)
        threshold_score = threshold_value
    threshold_value += step_factor
print('Best Threshold:', threshold_score, '--- Recall Score:', recall)

Threshold: 0.2 --- Recall Score: 0.9523809523809523
Threshold: 0.25 --- Recall Score: 0.9523809523809523
Threshold: 0.3 --- Recall Score: 0.9523809523809523
Threshold: 0.35 --- Recall Score: 0.9523809523809523
Threshold: 0.39999999999999997 --- Recall Score: 0.9523809523809523
Threshold: 0.44999999999999996 --- Recall Score: 0.9523809523809523
Threshold: 0.49999999999999994 --- Recall Score: 0.9523809523809523
Threshold: 0.5499999999999999 --- Recall Score: 0.9285714285714286
Threshold: 0.6 --- Recall Score: 0.9285714285714286
Threshold: 0.65 --- Recall Score: 0.9047619047619048
Threshold: 0.7000000000000001 --- Recall Score: 0.9047619047619048
Threshold: 0.7500000000000001 --- Recall Score: 0.8809523809523809
Threshold: 0.8000000000000002 --- Recall Score: 0.8809523809523809
Threshold: 0.8500000000000002 --- Recall Score: 0.8809523809523809
Threshold: 0.9000000000000002 --- Recall Score: 0.8571428571428571
Threshold: 0.9500000000000003 --- Recall Score: 0.8571428571428571
Best Thresho

Train with the full dataset without test split

In [134]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [135]:
# Scale the data
X_train = df.drop(columns='diagnosis')

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

y_train = df['diagnosis'] 

model = xgb.XGBClassifier(
        objective = 'binary:logistic',
        eval_metric = 'logloss',
        random_state = 42,
        scale_pos_weight= 2
    )
model.fit(X_train_scaled, y_train)

# Create a CalibratedClassifierCV
calibrated_model = CalibratedClassifierCV(model, method='sigmoid')
calibrated_model.fit(X_train_scaled, y_train, verbose=False)  # Fit the calibration

,estimator,"XGBClassifier...ree=None, ...)"
,method,'sigmoid'
,cv,None
,n_jobs,None
,ensemble,'auto'
,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None


In [136]:
# Save as a tuple
joblib.dump((scaler, model), "model.joblib")

['model.joblib']

In [137]:
# Save the calibrated model
joblib.dump(calibrated_model, 'calibrated_xgb_model.pkl')

['calibrated_xgb_model.pkl']

In [138]:
# Save the scaler
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [139]:
# Load the calibrated model using joblib
loaded_calibrated_model = joblib.load('calibrated_xgb_model.pkl')

In [140]:
# pdf_files = ["fig_1.pdf", "fig_3.pdf", "fig_2.pdf", "fig_metrics.pdf", "temp_fig_perimeter_mean.pdf",
#              "temp_fig_radius_mean.pdf", "temp_fig_texture_mean.pdf", "fig_perimeter_mean.pdf", "fig_radius_mean.pdf", "fig_texture_mean.pdf"]

# pdf_files.append(filename)

# # Merge all PDFs into one
# merger = PdfMerger()
# for pdf in pdf_files:
#     merger.append(pdf)

# output_file = "combined_figures.pdf"
# merger.write(output_file)
# merger.close()

# # Optional cleanup
# # for pdf in pdf_files:
# #     os.remove(pdf)

# print(f"Saved to {output_file}")

In [141]:
import webbrowser
webbrowser.open("combined_figures.pdf") 

True

In [142]:
X_train.columns

Index(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave_points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave_points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave_points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')